## Maximum Likelihood

### Learning objectives

By the end of this lesson, you will be familiar with 

* the maximum likelihood approach
* how to calculate the maximum likelihood of observing two sequences under the JC69 & TN93 models
* how to calculate the maximum likelihood of observing the sequence data when the parameters of tree and substitution model (e.g. TN93) are given

### The Maximum Likelihood Approach

In statistics, the likelihood is defined as the probability of observing the data when the parameters are given. The maximum likelihood estimation is a method of estimating the parameters for a model, given the observed data. The maximum likelihood estimation is intuitive and actually widely used. For example, one may 

In practice, the maximum likelihood estimation can be implemented by 

1) proposing a model, usually a distribution or a combination of multiple distributions, for the interested parameters.
2) creating a likelihood function. A likelihood function can be made up of many independent likelihood functions, where each describes the result of a trial relative to a statistical distribution. 
3) tuning parameters to maximize the likelihood function so that the observed data is most probable under the proposed model. This can be achieved by solving the derivative of the equation in some cases, or in most circumstances by trying many different values using a heuristic search.

In [650]:
# import packages and define additional functions
import numpy
import pandas
import toyplot
from IPython.display import display, Math
from sympy import Symbol, lambdify, diff, solve, latex, log, I, symbols, Matrix, diag, E, simplify
# from toytree import infer

# define the arbitrary nucleotide order throughout the notebook
BASE_ORDER = list("AGCT")


def eigendecomposed_matrices(input_matrix):
    """A sympy.Matrix.eigenvectors wrapper, 
    which use sympy.Matrix.eigenvectors to generate the matrix of eigenvectors and the diagonal matrix of eigenvalues

    Parameters
    ----------
    input_matrix : sympy.Matrix
        The input decomposible matrix

    Returns
    -------
    sympy.Matrix
        A square matrix whose ith column is the ith eigenvector of the input_matrix
    list
        A list whose ith element is ith eigenvalue of the input_matrix
    """
    eigenvals = []
    eigenvects = Matrix()
    for go_element, (eigen_val, eigen_multiplicity, eigen_space) in enumerate(input_matrix.eigenvects()):
        for eigen_vect in eigen_space:
            eigenvals.append(eigen_val)
            eigenvects = eigenvects.col_insert(go_element, eigen_vect)
    return eigenvects, eigenvals

### The Tosses Example

Let's consider a simple example of coin tosses, and estimating the heads probability. Let's assume that the tosses are all independent, and all have the same unknown heads probability,

In [402]:
heads_prob = Symbol("p")

basically, we are assuming this is a Binomial distribution (step 1), and the likelihood formula can be defined (step 2) as follows

In [406]:
def gen_binomial_likelihood_form(num_heads, num_tails, heads_prob):
    return heads_prob**num_heads * (1-heads_prob)**num_tails
# there is a binomial coefficient C before the formula, but it is a constant and therefore is always dropped in the likelihood calculation

In the felsenstein book chapter 16 (p249-250), observing the sequence of tosses `HHTTHTHHTTT`, we can calculate the probability of these data as

In [403]:
toss_likelihood_formula = gen_toss_likelihood_form(5, 6, heads_prob)
display(Math("L = Prob(D|p) = "+latex(toss_likelihood_formula)))

<IPython.core.display.Math object>

We can now plug in different values for parameters ($p$ in this case) to the likelihood formula and calculate the likelihood

In [503]:
# use lambdify to convert the likelihood formula to likelihood function in python
toss_likelihood_function = lambdify(heads_prob, toss_likelihood_formula)
# try p from 0 to 1 with 1000 values, call the likelihood function to calculate the likelihoods under different p values
toss_p = numpy.linspace(0, 1, num=1000)
toss_likelihood = toss_likelihood_function(toss_p)
toss_likelihood[:20]

array([0.00000000e+00, 9.98993994e-16, 3.17760974e-14, 2.39851221e-13,
       1.00465759e-12, 3.04752978e-12, 7.53757024e-12, 1.61934678e-11,
       3.13813405e-11, 5.62086745e-11, 9.46144304e-11, 1.51455386e-10,
       2.32588788e-10, 3.44951375e-10, 4.96634849e-10, 6.96957852e-10,
       9.56534525e-10, 1.28733970e-09, 1.70277082e-09, 2.21770660e-09])

Let's plot the likelihood $L$ against $p$

In [545]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian(xlabel="p", ylabel="Likelihood")
axes.y.ticks.labels.show = False
axes.x.ticks.locator = toyplot.locator.Uniform(count=6, format="{:.1f}")
likelihood_curve = axes.plot(toss_p, toss_likelihood)
#TODO: plot an arrow line
max_like_p = toss_p[toss_likelihood.argmax()]
max_like_line = axes.vlines(max_like_p, title=str(max_like_p))

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t444c1cfb9c87451b9829ac617cee754e"> <path d="M 50.0 250.0 L 50.20020020020021 249.99999999960903 L 50.4004004004004 249.99999998756365 L 50.6006006006006 249.99999990612844 L 50.8008008008008 249.99999960680304 L 51.00100100100101 249.99999880727577 L 51.2012012012012 249.99999704999018 L 51.40140140140141 249.99999366229602 L 51.6016016016016 249.99998771815592 L 51.8018018018018 249.99997800138027 L 52.002002002002 249.99996297036193 L 52.20220220220221 249.99994072428368 L 52.4024024024024 249.9999089707711 L 52.6026026026026 249.99986499496404 L 52.8028028028028 249.99980562998013 L 53.003003003003 249.99972722874392 L 53.2032032032032 249.99962563715593 L 53.40340340340341 249.9994961685756 L 53.6036036036036 249.99933357959296 L 53.8038038038038 249.99913204706388 L 54.004004004004 249.99888514638388 L 54.20420420420421 249.9985858309763 L 54.4044044044044 249.99822641297015 L 54.6046046046046 249.99779854504385 L 54.8048048048048 249.9972932034114 L 55.00500500500501 249.99670067192685 L 55.2052052052052 249.99601052728488 L 55.40540540540541 249.99521162529354 L 55.605605605605604 249.9942920881973 L 55.805805805805804 249.9932392930279 L 56.006006006006004 249.99203986096035 L 56.20620620620621 249.99067964765325 L 56.406406406406404 249.98914373455096 L 56.606606606606604 249.98741642112662 L 56.806806806806804 249.98548121804532 L 57.007007007007005 249.98332084122586 L 57.207207207207205 249.98091720678136 L 57.40740740740741 249.9782514268179 L 57.607607607607605 249.9753038060715 L 57.807807807807805 249.97205383936299 L 58.008008008008005 249.96848020985237 L 58.20820820820821 249.96456078807216 L 58.408408408408405 249.96027263172132 L 58.608608608608606 249.955591986201 L 58.808808808808806 249.95049428587308 L 59.00900900900901 249.94495415602358 L 59.20920920920921 249.9389454155127 L 59.409409409409406 249.93244108009316 L 59.609609609609606 249.92541336638044 L 59.809809809809806 249.9178336964558 L 60.01001001001001 249.90967270308641 L 60.210210210210214 249.90090023554458 L 60.410410410410414 249.8914853660102 L 60.61061061061061 249.88139639653897 L 60.810810810810814 249.87060086658101 L 61.01101101101101 249.8590655610331 L 61.211211211211214 249.8467565188092 L 61.41141141141141 249.83363904191333 L 61.61161161161161 249.81967770499924 L 61.81181181181181 249.80483636540208 L 62.012012012012015 249.78907817362702 L 62.212212212212215 249.7723655842792 L 62.412412412412415 249.75466036742205 L 62.61261261261261 249.73592362034768 L 62.812812812812815 249.71611577974673 L 63.01301301301302 249.69519663426314 L 63.21321321321321 249.6731253374195 L 63.41341341341341 249.6498604209004 L 63.61361361361361 249.62535980817992 L 63.81381381381381 249.59958082847993 L 64.01401401401401 249.57248023104646 L 64.21421421421422 249.54401419973138 L 64.41441441441441 249.51413836786648 L 64.61461461461461 249.4828078334181 L 64.81481481481481 249.4499771744093 L 65.01501501501501 249.41560046459833 L 65.21521521521521 249.37963128940112 L 65.41541541541541 249.34202276204596 L 65.61561561561561 249.30272753994944 L 65.81581581581581 249.26169784130178 L 66.01601601601601 249.21888546185087 L 66.21621621621621 249.17424179187358 L 66.41641641641641 249.12771783332403 L 66.61661661661661 249.07926421714814 L 66.81681681681681 249.0288312207535 L 67.01701701701703 248.9763687856253 L 67.21721721721721 248.92182653507714 L 67.41741741741741 248.8651537921274 L 67.61761761761761 248.80629959749174 L 67.81781781781783

The maximum likelihood was achieve (step 3) at 

In [546]:
display(Math("p=%f"%max_like_p))

<IPython.core.display.Math object>

This can be verified by taking the derivative of $L$ with respect to $p$, equating it to zero, and solving:

In [413]:
# use diff() to calculate the derivative
toss_like_derivatives = diff(toss_likelihood_formula, heads_prob)
# display the derivative
display(Math("\dfrac{dL}{dp} = "+latex(toss_like_derivatives)+" = 0"))
# solve
extreme_values = np.array(solve(toss_like_derivative, heads_prob))
extreme_likelihood = toss_likelihood_function(extreme_values)
display(Math(f"p={extreme_values[extreme_likelihood.argmax()]}"))

<IPython.core.display.Math object>

In [138]:
# TODO: logarithm

In [432]:
def gen_binomial_loglike_form(num_heads, num_tails, heads_prob):
    return num_heads * log(heads_prob) + num_tails * log(1-heads_prob)

### From substitution rate to probability and likelihood

Now let's move on to the sequences and use maximum likelihood to estimate the phylogenetic distance between two sequences.
Supposing we have following alignment of two sequences:

In [338]:
# TODO: make it look pretty

In [495]:
# I'm picking this to be compatible with text book to confirm, it can be randomly generated
Bulbasaur="T"*(179+23+1+0)+"C"*(30+219+2+0)+"A"*(2+1+291+10)+"G"*(0+0+21+169)
Venusaur=("T"*179+"C"*23+"A"*1)+("T"*30+"C"*219+"A"*2)+("T"*2+"C"*1+"A"*291+"G"*10)+("A"*21+"G"*169)

with following matrix of observed changes

In [560]:
changes_count_matrix = pandas.DataFrame(np.zeros((4,4)), columns=BASE_ORDER, index=BASE_ORDER, dtype=int)
for go_base, bulb_base in enumerate(Bulbasaur):
    venu_base = Venusaur[go_base]
    changes_count_matrix[venu_base][bulb_base] += 1
changes_count_matrix

,A,G,C,T
A,291,10,1,2
G,21,169,0,0
C,2,0,219,30
T,1,0,23,179


Now we have the observed data, let's look at the models.

Note: all the models we used here are assuming that all DNA sites in different lineages are independent and identically distributed (i.i.d.). This includes the models for heterogeneity among sites (The gamma model). While the model allows different sites to evlove at different rates, it does not specify a _priori_ which sites should have which rates (Yang 2014 Chapter 4).

#### Example: The JC69 model
The Jukes-Cantor model assumes that every nucleotide has the same substitution rate (denoted as $\mu$) of changing into every other nucleotide, so that we can have the rate matrix ($Q$)

In [389]:
# define the substitution rate
mu = Symbol("mu")
# the substitution rate matrix under the JC69 model
Q_JC69 = Matrix([[-3*mu, mu, mu, mu],
                 [mu, -3*mu, mu, mu],
                 [mu, mu, -3*mu, mu],
                 [mu, mu, mu, -3*mu]])
display(Math("Q=\{q_{ij}\}="+latex(Q_JC69)))

<IPython.core.display.Math object>

>In-class Question: what is the substitution rate for the sequence given this rate matrix?


The rate matrix is determining the Markov process. To fit our observed sequences into our likelihood framework, we need to know the probability matrix under which base $i$ becomes base $j$ after a certain time span ($t$). This is usually called transition probability (note to distinguish the transition in Markove models and in biology). A simplest calculation is assuming time span $t$ to be very small so that we have $p_{ij}\approx{t{q_{ij}}}$ for ${i}\neq{j}$, and $p_{ij}\approx{1-t\sum_{i\neq{j}}q_{ij}}$ for ${i}={j}$. The probability matrix is thus

In [325]:
# define the branch length
timespan = Symbol("t")
display(Math("P(t) = \{p_{ij}(t)\} \\approx I+Qt = " + latex(diag(1,1,1,1)+Q_JC69*timespan)))

<IPython.core.display.Math object>

In the real case, the time span $t$ is barely that small, so we have

In [326]:
display(Math("P(t) = e^{Qt}"))

<IPython.core.display.Math object>

The calculation of this matrix exponential can be done using the eigendecomposition (or spectral decomposition). As soon as $Q$ is a diagonalizable matrix, it can be factorized as

In [334]:
m_theta, list_lambda = eigendecomposed_matrices(Q_JC69)
display(Math("Q = \Theta\Lambda\Theta^{-1} = " + latex(m_theta) + latex(diag(*list_lambda)) + latex(m_theta**(-1))))

<IPython.core.display.Math object>

where $\Theta$ is a square matrix whose $i$th column is the $i$th eigenvector of $Q$, and $\Lambda$ is a diagonal matrix whose $i$th diagonal element ($\lambda_i$) is the $i$th eigenvalue of $Q$. There is a super cool property of this decomposed expression that any algebraic function $h$ of matrix $Q$ can be calculated as $h(Q) = \Theta h(\Lambda) \Theta^{-1} = \Theta diag\{h(\lambda)\} \Theta^{-1}$. So we have

In [565]:
m_lambda = diag(*[E**(_eigenval_*timespan) for _eigenval_ in list_lambda])
P_JC69 = m_theta*m_lambda*m_theta.inv()
display(Math("P(t) = e^{Qt} = " + latex(m_theta) + latex(m_lambda) + latex(m_theta**(-1)) + " = " + latex(P_JC69)))

<IPython.core.display.Math object>

i.e.

$P(t) = e^{Qt} = 
\left[\begin{array}{cc}
p_0(t) & p_1(t) & p_1(t) & p_1(t)\\
p_1(t) & p_0(t) & p_1(t) & p_1(t)\\
p_1(t) & p_1(t) & p_0(t) & p_1(t)\\
p_1(t) & p_1(t) & p_1(t) & p_0(t)\\
\end{array}\right]$, with 
$\begin{cases} p_0(t)=\frac{1}{4} + \frac{3 e^{- 4 \mu t}}{4} \\ p_1(t)=\frac{1}{4} - \frac{e^{- 4 \mu t}}{4} \end{cases}$

Note that in the probability matrix, the substituion rate $\mu$ and time span $t$ are awalys in the form of their product $\mu t$. Without external information about $\mu$ (e.g. using rates from previous studies) or $t$ (e.g. using fossil calibrations), we can not tell whether the differences between sequences are caused by a higher rate over a short time span, or by a slow rate over a long time span. The likelihood estimation will be the same for any combination of $\mu$ and $t$ as long as $\mu t$ is fixed.

Also note that in the Q matrix, the total substitution rate of any nucleotide is $3\mu$. So we can replace $\mu$ and $t$ with its product, the phylogenetic distance $d$, given $d=3\mu t$

In [527]:
# replace mu*timespan with dist
dist = Symbol("d")
P_JC69 = P_JC69.subs(mu*timespan, dist/3)

Now we know that under the JC69 model, the probability of observing different nucleotides at a site between two sequences is 

In [529]:
display(Math("p=3p_1=" + latex(3*P_JC69[0,1])))

<IPython.core.display.Math object>

Given that all DNA sites in different lineages are assumed to be i.i.d. and that there are only two observed states in our data (matched vs unmatched), we can fit the data to the binomial distribution. The binomial distribution is then the real model we are using in our likelihood function formulation (the first step). The likelihood of observing $x$ mismatched sites out of $n$ sites, like the coin toss, is given by

In [530]:
num_mismatch, num_sites = symbols(["x", "n"])
jc_2seq_like_formula = gen_binomial_likelihood_form(num_heads=num_sites-num_mismatch, num_tails=num_mismatch, heads_prob=P_JC69[0,0])
display(Math("L(d;x) = " + latex(jc_2seq_like_formula)))

<IPython.core.display.Math object>

with the log form as

In [531]:
jc_2seq_loglike_formula = gen_binomial_loglike_form(num_heads=num_sites-num_mismatch, num_tails=num_mismatch, heads_prob=P_JC69[0,0])
display(Math("\mathscr{l}(d;x) = " + latex(jc_2seq_loglike_formula)))

<IPython.core.display.Math object>

Now let's come back to the sequences of Bulbasaur and Venusaur. We observed $90$ mismathed sites (summing over the non-diagonal elements) out of $948$ sites, and plug in the observed data into the log-likelihood formula $\mathscr{l}$ to get

In [532]:
# convert to numpy.arrary
changes_count_array = np.array(changes_count_matrix)
# number of total sites
n_sites = changes_count_array.sum()
# sum over the diagonal elements to get the number of matched sites
n_match=changes_count_array.diagonal().sum()
# number of mismatched sites
n_mismatch = n_sites - n_match
# plug in the observed data
jc_2seq_loglike_case_formula = gen_binomial_loglike_form(num_heads=n_match, num_tails=n_mismatch, heads_prob=P_JC69[0,0])
display(Math("\mathscr{l}(d) = " + latex(jc_2seq_loglike_case_formula)))

<IPython.core.display.Math object>

This is the likelihood function. We are going to tune our only parameter, the phylogenetic distance $d$, to maximize it. 

Similarly, we may try the numbers, 

In [533]:
# use lambdify to convert the likelihood formula to likelihood function in python
jc_2seq_loglike_case_function = lambdify(dist, jc_2seq_loglike_case_formula)
# try d from 0 to 1 with 1000 values, call the likelihood function to calculate the likelihoods under different BV_dist values
BV_dist = numpy.linspace(1e-7, 1, num=1000)
jc_2seq_loglike = jc_2seq_loglike_case_function(BV_dist)
jc_2seq_loglike[:20]

array([-1450.62870033,  -622.51780913,  -561.05752671,  -525.48533378,
        -500.51256434,  -481.34766142,  -465.85628727,  -452.89990534,
        -441.79890448,  -432.1149186 ,  -423.5486329 ,  -415.88655741,
        -408.97105853,  -402.68242718,  -396.92761139,  -391.63284546,
        -386.73866292,  -382.19642393,  -377.9658368 ,  -374.01315056])

In [547]:
canvas = toyplot.Canvas(width=900, height=300)
axes = canvas.cartesian(xlabel="distance (subs/site)", ylabel="Likelihood")
axes.y.ticks.labels.show = False
axes.x.ticks.locator = toyplot.locator.Uniform(count=21, format="{:.2f}")
likelihood_curve = axes.plot(toss_p, jc_2seq_loglike)
#TODO: plot an arrow line
max_like_dist = BV_dist[jc_2seq_loglike.argmax()]
max_like_line = axes.vlines(max_like_dist, title=str(max_like_dist))

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="900.0px" height="300.0px" viewBox="0 0 900.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t58eeb7d41829477f8dc2fc81a5591e8b"> <path d="M 50.0 241.78858746427812 L 50.80080080080081 104.05755347697644 L 51.6016016016016 93.83550672492913 L 52.4024024024024 87.91915546590725 L 53.2032032032032 83.76569559366023 L 54.00400400400401 80.57819748947566 L 54.8048048048048 78.00167905201252 L 55.60560560560561 75.8467793899191 L 56.406406406406404 74.00046587827664 L 57.207207207207205 72.38982966738041 L 58.008008008008005 70.96508885079618 L 58.80880880880881 69.69073587487806 L 59.609609609609606 68.54055318574078 L 60.41041041041041 67.49463082565148 L 61.21121121121121 66.53749242971536 L 62.01201201201201 65.65686932287636 L 62.812812812812815 64.8428710660799 L 63.613613613613616 64.08740790312186 L 64.41441441441441 63.38377854033208 L 65.21521521521521 62.726369526992976 L 66.01601601601601 62.11043183479737 L 66.81681681681681 61.53191200438908 L 67.61761761761761 60.98732260843729 L 68.41841841841841 60.47364153279405 L 69.21921921921921 59.98823270896365 L 70.02002002002003 59.5287830386539 L 70.82082082082081 59.09325169661788 L 71.62162162162163 58.67982900652138 L 72.42242242242241 58.286902799362664 L 73.22322322322321 57.91303067790277 L 74.02402402402402 57.556916984988284 L 74.82482482482483 57.21739354976481 L 75.62562562562563 56.89340349171672 L 76.42642642642642 56.583987517667154 L 77.22722722722722 56.28827226498403 L 78.02802802802802 56.005460334938675 L 78.82882882882883 55.734821730413714 L 79.62962962962963 55.47568646699837 L 80.43043043043043 55.22743816964457 L 81.23123123123122 54.989508501224925 L 82.03203203203203 54.76137229657607 L 82.83283283283284 54.542543297472236 L 83.63363363363362 54.332570401614696 L 84.43443443443442 54.131034353045564 L 85.23523523523524 53.93754481308029 L 86.03603603603604 53.751737760435844 L 86.83683683683684 53.57327317713183 L 87.63763763763764 53.401832983279625 L 88.43843843843844 53.237119189312956 L 89.23923923923923 53.07885223875206 L 90.04004004004004 52.926769518401656 L 90.84084084084085 52.78062401608379 L 91.64164164164164 52.640183108713615 L 92.44244244244243 52.50522746581858 L 93.24324324324326 52.375550055550654 L 94.04404404404404 52.250955241906304 L 94.84484484484484 52.13125796329172 L 95.64564564564564 52.01628298379306 L 96.44644644644644 51.905864209563866 L 97.24724724724724 51.79984406364975 L 98.04804804804805 51.69807291335664 L 98.84884884884886 51.60040854495025 L 99.64964964964966 51.50671568106917 L 100.45045045045043 51.416865536749846 L 101.25125125125126 51.33073541041444 L 102.05205205205206 51.24820830656955 L 102.85285285285285 51.16917258730931 L 103.65365365365365 51.093521650024854 L 104.45445445445445 51.02115362898974 L 105.25525525525525 50.95197111872953 L 106.05605605605606 50.88588091729535 L 106.85685685685687 50.82279378774677 L 107.65765765765767 50.76262423631561 L 108.45845845845847 50.70529030587063 L 109.25925925925927 50.65071338343354 L 110.06006006006007 50.59881802061425 L 110.86086086086087 50.54953176593933 L 111.66166166166167 50.50278500814014 L 112.46246246246247 50.458510829552196 L 113.26326326326325 50.41664486885311 L 114.06406406406407 50.377125192433844 L 114.86486486486487 50.33989217376036 L 115.66566566566567 50.30488838013632 L 116.46646646646647 50.272058466329895 L 117.26726726726726 50.241349074570095 L 118.06806806806807 50.21270874046133 L 118.86886886886886 50.186087804400124 L 119.66966966966966 50.16143832811277 L 120.47047047047047 50.13871401596253 L 121.27127127127127 50

The maximum likelihood was achieve (step 3) at 

In [548]:
display(Math("p=%f"%max_like_dist))

<IPython.core.display.Math object>

In [549]:
# TODO generate heuristic search and a plot

Like in the tosses example, you may maximize $\mathscr{l}$ by setting $\mathrm{d}\mathscr{l}/\mathrm{d}d=0$

In [550]:
# use diff() to calculate the derivative over dist
jc_2seq_loglike_derivatives = diff(jc_2seq_loglike_formula, dist)
# display the derivative
display(Math("\dfrac{\mathrm{d}\mathscr{l}}{\mathrm{d}d} = "+latex(jc_2seq_loglike_derivatives)+" = 0"))

<IPython.core.display.Math object>

so the $\mathscr{l}$ is maximized at

In [551]:
# solve
extreme_values = np.array(solve(jc_2seq_loglike_derivatives, dist))
# there are four roots, let's exclude the three roots that is a complex number when e.g. num_mismatch=2, num_sites=10
real_values = []
for extreme_val in extreme_values:
    if not extreme_val.has(I) and not extreme_val.subs(num_mismatch, 2).subs(num_sites, 10).has(I):
        real_values.append(simplify(extreme_val))
# display the only solution
display(Math("\hat{d}=" + latex(real_values[0])))

<IPython.core.display.Math object>

Then plug in the sequence alignment of Bulbasaur and Venusaur, we have the analytical solution as

In [553]:
BV_dist_analytical = float(simplify(real_values[0].subs(num_sites, 948).subs(num_mismatch, 90)))
display(Math("\hat{d}=" + str(BV_dist_analytical)))

<IPython.core.display.Math object>

#### Example: The TN93 model (unfinished)

In [576]:
# define the equilibrium base frequencies
pi_A, pi_G, pi_C, pi_T = symbols(["pi_" + _base for _base in BASE_ORDER])
# define the transition and transversion rates
alpha_1, alpha_2, beta = symbols(["alpha_1", "alpha_2", "beta"])
# define the substitution rate matrix 
Q_TN93 = Matrix([[-(pi_G*alpha_1+pi_C*beta+pi_T*beta), pi_A*alpha_1, pi_A*beta, pi_A*beta],
                 [pi_G*alpha_1, -(pi_A*alpha_1+pi_C*beta+pi_T*beta), pi_G*beta, pi_G*beta],
                 [pi_C*beta, pi_C*beta, -(pi_A*beta+pi_G*beta+pi_T*alpha_2), pi_C*alpha_2],
                 [pi_T*beta, pi_T*beta, pi_T*alpha_2, -(pi_A*beta+pi_G*beta+pi_C*alpha_2)]])
Q_TN93

Matrix([
[-alpha_1*pi_G - beta*pi_C - beta*pi_T,                          alpha_1*pi_A,                             beta*pi_A,                             beta*pi_A],
[                         alpha_1*pi_G, -alpha_1*pi_A - beta*pi_C - beta*pi_T,                             beta*pi_G,                             beta*pi_G],
[                            beta*pi_C,                             beta*pi_C, -alpha_2*pi_T - beta*pi_A - beta*pi_G,                          alpha_2*pi_C],
[                            beta*pi_T,                             beta*pi_T,                          alpha_2*pi_T, -alpha_2*pi_C - beta*pi_A - beta*pi_G]])

Now, unlike the JC69 model, the substitution rates are different among four different nucleotides.

>In-class Question: What is the substitution rate for the sequence given this rate matrix?

Considering the substitution process in equilibrium, remember that we had this nice figure for a random walk in a Markov chain of 4 states in our previous class.

<div class="markov">
  <iframe width="800" height="500" src="https://setosa.io/markov/index.html#%7B%22tm%22%3A%5B%5B0.5%2C0.2%2C0.1%2C0.2%5D%2C%5B0.1%2C0.5%2C0.3%2C0.1%5D%2C%5B0.05%2C0.2%2C0.7%2C0.05%5D%2C%5B0%2C0.05%2C0.05%2C0.9%5D%5D%7D"></iframe>
</div>

We can imagine, that the amount of time the Markov chain spends in each nucleotide is proportional to its equilibrium frequency. In other words, for each site, each nucleotide contributes to the average substitution rate by its equilibrium frequency. Thus the average substitution rate is the sum of the substitution rate of a nucleotide times its equilibrium frequency, given by

In [631]:
rate_single_row = Q_TN93.diagonal()
pi_single_col = Matrix([pi_A, pi_G, pi_C, pi_T])
# the only element in the matrix
mu_res = (rate_single_row*pi_single_col)[0]
display(Math("\mu=\sum_i {pi_i q_{ii}}=" + latex(mu_res)))
# one may also confirm this by calculating the JC69 model rate, simplify(Q_JC69.diagonal()*Matrix([1/4,1/4,1/4,1/4]))

<IPython.core.display.Math object>

Instead of step-by-step calculation, we may also achieve the probability matrix directly using the matrix exponential calculation of sympy

In [566]:
# the probability matrix can also directly achieved using sympy with following code
P_TN93 = (Q_TN93*timespan).exp()
P_TN93

Matrix([
[pi_A/(pi_A + pi_C + pi_G + pi_T) + pi_G*exp(-alpha_1*pi_A*t - alpha_1*pi_G*t - beta*pi_C*t - beta*pi_T*t)/(pi_A + pi_G) + pi_T*(pi_A*pi_C + pi_A*pi_T)*exp(-beta*pi_A*t - beta*pi_C*t - beta*pi_G*t - beta*pi_T*t)/((pi_A*pi_T + pi_G*pi_T)*(pi_A + pi_C + pi_G + pi_T)), pi_A/(pi_A + pi_C + pi_G + pi_T) - pi_A*exp(-alpha_1*pi_A*t - alpha_1*pi_G*t - beta*pi_C*t - beta*pi_T*t)/(pi_A + pi_G) + pi_T*(pi_A*pi_C + pi_A*pi_T)*exp(-beta*pi_A*t - beta*pi_C*t - beta*pi_G*t - beta*pi_T*t)/((pi_A*pi_T + pi_G*pi_T)*(pi_A + pi_C + pi_G + pi_T)),                                                                                                  pi_A/(pi_A + pi_C + pi_G + pi_T) - (pi_A*pi_C + pi_A*pi_T)*exp(-beta*pi_A*t - beta*pi_C*t - beta*pi_G*t - beta*pi_T*t)/(pi_A*pi_C + pi_A*pi_T + pi_C**2 + pi_C*pi_G + 2*pi_C*pi_T + pi_G*pi_T + pi_T**2),                                                                                                     pi_A/(pi_A + pi_C + pi_G + pi_T) - (pi_A*pi_C + pi_A*pi_T)*

# TODO
Usually we replace alpha ... kappa
Now we implemented this in toytree

In [637]:
from toytree.infer import *
tn93 = TN93()

In [638]:
tn93.p_matrix()

Matrix([
[(pi_C*(pi_A + pi_C + pi_G + pi_T)*exp(d*(kappa_1*pi_C + kappa_1*pi_T + pi_A + pi_G)/(2*(kappa_1*pi_C*pi_T + kappa_2*pi_A*pi_G + pi_A*pi_C + pi_A*pi_T + pi_C*pi_G + pi_G*pi_T))) + pi_T*(pi_A + pi_G)*exp(d*(pi_A + pi_C + pi_G + pi_T)/(2*(kappa_1*pi_C*pi_T + kappa_2*pi_A*pi_G + pi_A*pi_C + pi_A*pi_T + pi_C*pi_G + pi_G*pi_T))) + pi_T*(pi_C + pi_T))/((pi_C + pi_T)*(pi_A + pi_C + pi_G + pi_T)),             pi_T*(pi_C + pi_T + (pi_A + pi_G)*exp(d*(pi_A + pi_C + pi_G + pi_T)/(2*(kappa_1*pi_C*pi_T + kappa_2*pi_A*pi_G + pi_A*pi_C + pi_A*pi_T + pi_C*pi_G + pi_G*pi_T))) - (pi_A + pi_C + pi_G + pi_T)*exp(d*(kappa_1*pi_C + kappa_1*pi_T + pi_A + pi_G)/(2*(kappa_1*pi_C*pi_T + kappa_2*pi_A*pi_G + pi_A*pi_C + pi_A*pi_T + pi_C*pi_G + pi_G*pi_T))))/((pi_C + pi_T)*(pi_A + pi_C + pi_G + pi_T)),                                                                                                                                                                                                               

>In-class Question: Please compare the likelihood values generated by the JC69 model and the TN93 model, why is the likelihood value by TN93 even much lower?

#### The GTR model

GTR model do not have a formularized solution. But it can be solved case by case. Please refer to Felsenstein's solution.

In [555]:
# define the rates
A_to_G, A_to_C, A_to_T, G_to_C, G_to_T, C_to_T,  = symbols(["alpha", "beta", "gamma", "delta", "epsilon", "eta"])
# define the substitution rate matrix (the tranpose form; a_ij is the rate at which base j changes into base i)
Q_GTR = Matrix([[-(pi_G*A_to_G+pi_C*A_to_C+pi_T*A_to_T), pi_A*A_to_G, pi_A*A_to_C, pi_A*A_to_T],
                [pi_G*A_to_G, -(pi_A*A_to_G+pi_C*G_to_C+pi_T*G_to_T), pi_G*G_to_C, pi_G*G_to_T],
                [pi_C*A_to_C, pi_C*G_to_C, -(pi_A*A_to_C+pi_G*G_to_C+pi_T*C_to_T), pi_C*C_to_T],
                [pi_T*A_to_T, pi_T*G_to_T, pi_T*C_to_T, -(pi_A*A_to_T+pi_G*G_to_T+pi_C*C_to_T)]])
Q_GTR

Matrix([
[-alpha*pi_G - beta*pi_C - gamma*pi_T,                              alpha*pi_A,                          beta*pi_A,                            gamma*pi_A],
[                          alpha*pi_G, -alpha*pi_A - delta*pi_C - epsilon*pi_T,                         delta*pi_G,                          epsilon*pi_G],
[                           beta*pi_C,                              delta*pi_C, -beta*pi_A - delta*pi_G - eta*pi_T,                              eta*pi_C],
[                          gamma*pi_T,                            epsilon*pi_T,                           eta*pi_T, -epsilon*pi_G - eta*pi_C - gamma*pi_A]])

### Computing the likelihood of a tree

In [651]:
# import new version to calculate the likelihood of a tree

In [652]:
from toytree.infer import JC69

ModuleNotFoundError: No module named 'toytree.infer'

In [2]:
from typing import Optional, Collection
from loguru import logger
import numpy as np
import toytree

In [49]:
# TODO: this can be generated using simulation

tree = toytree.tree("((sp-A:0.05,sp-B:0.06):0.03,(((sp-C:0.01,sp-D:0.01):0.03,sp-E:0.05):0.01,sp-F:0.04):0.02);")

In [48]:
# can be randomly generated
rng = np.random.default_rng(12345)
characters = {f'sp-{chr(65+tip_n)}':rng.choice(list("ATGC")) for tip_n in range(6)}
characters

{'sp-A': 'G', 'sp-B': 'A', 'sp-C': 'C', 'sp-D': 'T', 'sp-E': 'A', 'sp-F': 'C'}

In [51]:
# TODOoptimize plot
tree.draw(node_labels=True, node_sizes=16)

(<toyplot.canvas.Canvas at 0x123425e40>,
 <toytree.Render.ToytreeMark at 0x123425cc0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="270.0px" height="275.0px" viewBox="0 0 270.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4489529415fa4aeebc7bacafe36f8e9d"> 0 1 2 3 4 5 6 7 8 9 10 sp-D sp-C sp-E sp-F sp-B sp-A

#### Pruning algorithm of Felsenstein
Nesting rule or Horner’s rule

#### We will cover searching for ML tree, Variable site rates, Gaps, etc in the next class